In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt            # library for visualization
import matplotlib
import re
from collections import Counter
from nltk import ngrams
from nltk.tokenize import TweetTokenizer
from nltk.stem.isri import ISRIStemmer
%matplotlib inline 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import json
from sklearn.feature_extraction import DictVectorizer


In [ ]:
!gdown --id 1aT120TILDKNsqMbV9w-APglATaMJ3lKY

Downloading...
From: https://drive.google.com/uc?id=1aT120TILDKNsqMbV9w-APglATaMJ3lKY
To: /content/webnlg-dataset-master.zip
100% 12.4M/12.4M [00:00<00:00, 75.2MB/s]


In [ ]:
!unzip /content/webnlg-dataset-master.zip

In [2]:
import re
  
def camel_case_split(strs):
    words = [[strs[0]]]
  
    for c in strs[1:]:
        if words[-1][-1].islower() and c.isupper():
            words.append(list(c))
        else:
            words[-1].append(c)
  
    return " ".join([''.join(word) for word in words])
def replace_(strs):

  return strs.replace('_'," ")

def handelstr(strs):
  r=camel_case_split(strs)
  r=replace_(r)
  r=r.lower()
  return r
      
# Driver code
strs = "GeeksForGeeks"
print(handelstr(strs).split())

['geeks', 'for', 'geeks']


In [4]:

with open('C:/Users/Fares_i9bkpvz/Desktop/data/webnlg-dataset-master/release_v2/json/webnlg_release_v2_train.json', encoding="utf8") as f:
    train = json.load(f)
with open('C:/Users/Fares_i9bkpvz/Desktop/data/webnlg-dataset-master/release_v2/json/webnlg_release_v2_test.json', encoding="utf8") as f:
    test = json.load(f)
with open('C:/Users/Fares_i9bkpvz/Desktop/data/webnlg-dataset-master/release_v2/json/webnlg_release_v2_dev.json', encoding="utf8") as f:
    dev = json.load(f)

In [5]:
def relexicalize(data,s,i):
  e=s
  category=handelstr(data['entries'][i][str(i+1)]['category'])
  for j in range(len(data['entries'][i][str(i+1)]['modifiedtripleset'])):
      
      objectt=data['entries'][i][str(i+1)]['modifiedtripleset'][j]['object']
      propertyy=handelstr(data['entries'][i][str(i+1)]['modifiedtripleset'][j]['property'])
      subject=data['entries'][i][str(i+1)]['modifiedtripleset'][j]['subject']
      e=e.replace(category.lower()+'k',subject)
      e=e.replace(propertyy.lower()+'k',objectt)

  return e

In [6]:
# 387 test 1
test['entries'][388]

{'389': {'category': 'Airport',
  'lexicalisations': [{'comment': 'good',
    'lex': 'Aarhus Airport, operated by Aarhus Lufthavn A/S, has a runway length of 2777.0.',
    'xml_id': 'Id1'},
   {'comment': 'good',
    'lex': 'Aarhus Lufthavn A/S is the operation organisation of Aarhus Airport which has a runway length of 2777.',
    'xml_id': 'Id2'}],
  'modifiedtripleset': [{'object': '2777.0',
    'property': 'runwayLength',
    'subject': 'Aarhus_Airport'},
   {'object': '"Aarhus Lufthavn A/S"',
    'property': 'operatingOrganisation',
    'subject': 'Aarhus_Airport'}],
  'originaltriplesets': {'originaltripleset': [[{'object': '"2777.0"^^xsd:double',
      'property': 'runwayLength',
      'subject': 'Aarhus_Airport'},
     {'object': '"Aarhus Lufthavn A/S"@en',
      'property': 'operator',
      'subject': 'Aarhus_Airport'}]]},
  'shape': '(X (X) (X))',
  'shape_type': 'sibling',
  'size': '2',
  'xml_id': 'Id1'}}

In [7]:
def get_delexilized_data(data,k=-1):
  y=[]
  x=[]
  if k==-1:
    k=len(data['entries'])
    print(k)
  for i in range(k):
      tar=data['entries'][i][str(i+1)]['lexicalisations'][0]['lex']
      tar=handelstr(tar)
      olist=[]
      category=data['entries'][i][str(i+1)]['category']
      category=handelstr(category)
      for j in range(len(data['entries'][i][str(i+1)]['modifiedtripleset'])):
          
          objectt=handelstr(data['entries'][i][str(i+1)]['modifiedtripleset'][j]['object'])
          propertyy=handelstr(data['entries'][i][str(i+1)]['modifiedtripleset'][j]['property'])
          subject=handelstr(data['entries'][i][str(i+1)]['modifiedtripleset'][j]['subject'])
          tar=tar.replace(subject,category.upper()+'k')
          tar=tar.replace(objectt,propertyy.upper()+'k')
          olist.append(category.upper()+'k')
          olist.append(propertyy)
          olist.append(propertyy.upper()+'k')
      y.append(tar)    
      x.append(olist)
  return [x,y]


def get_data(data,k=-1):
  y=[]
  x=[]
  if k==-1:
    k=len(data['entries'])
    print(k)
  for i in range(k):
      y.append(data['entries'][i][str(i+1)]['lexicalisations'][0]['lex'])
      olist=[]
      for j in range(len(data['entries'][i][str(i+1)]['modifiedtripleset'])):       
          objectt=data['entries'][i][str(i+1)]['modifiedtripleset'][j]['object']
          propertyy=data['entries'][i][str(i+1)]['modifiedtripleset'][j]['property']
          subject=data['entries'][i][str(i+1)]['modifiedtripleset'][j]['subject']
          olist.append(objectt)
          olist.append(propertyy)
          olist.append(subject)
      x.append(olist)
  return [x,y]

In [8]:
x,y=get_delexilized_data(train)
xtest,ytest=get_delexilized_data(test)
xdev,ydev=get_delexilized_data(dev)


12876
1600
1619


In [9]:
xdev[0]
# train['entries'][909][str(909+1)]['lexicalisations'][0]['lex']

['AIRPORTk', 'leader name', 'LEADER NAMEk']

In [10]:
ydev[0]

'the leader of AIRPORTk is LEADER NAMEk.'

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip 'glove.6B.zip' -d 'data'

--2022-01-05 16:15:01--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-01-05 16:15:02--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.95MB/s    in 2m 42s  

2022-01-05 16:17:44 (5.09 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: data/glove.6B.50d.txt   
  inflating: data/glove.6B.100d.txt  
  inflating: 

In [11]:
#Add sostok and eostok at 
for i in range(len(y)):
    y[i]='sostok '+ y[i] + ' eostok'
    
y[0]

'sostok the aarhus is the airport of aarhus, denmark. eostok'

In [12]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='UNK')
tokenizer.fit_on_texts(x)
X_train1 = tokenizer.texts_to_sequences(x)
xtest1 = tokenizer.texts_to_sequences(xtest)
xdev1 = tokenizer.texts_to_sequences(xdev)

tk = Tokenizer(oov_token='UNK')
tk.fit_on_texts(y)
y_train1 = tk.texts_to_sequences(y)
ytest1 = tk.texts_to_sequences(ytest)
ydev1 = tk.texts_to_sequences(ydev)



In [13]:
lengthey = []
lengthex = []
for i in range(len(X_train1)):
    lengthex.append(len(X_train1[i]))

for i in range(len(y_train1)):
    lengthey.append(len(y_train1[i]))

MAX_LENGTHy=max(lengthey)
MAX_LENGTHx=max(lengthex)

In [14]:
xtest1[0]

[7, 60, 61]

In [15]:
vocab_sizex = len(tokenizer.word_index) + 1
print(vocab_sizex)

vocab_sizey = len(tk.word_index) + 1
print(vocab_sizey)

762
5001


In [16]:
from keras.preprocessing.sequence import pad_sequences

y_train1 = pad_sequences(y_train1, maxlen=MAX_LENGTHy, padding='post')
X_train1 = pad_sequences(X_train1, maxlen=MAX_LENGTHx, padding='post')
xtest1 = pad_sequences(xtest1, maxlen=MAX_LENGTHx, padding='post')
ytest1 = pad_sequences(ytest1, maxlen=MAX_LENGTHy, padding='post')
xdev1 = pad_sequences(xdev1, maxlen=MAX_LENGTHx, padding='post')
ydev1 = pad_sequences(ydev1, maxlen=MAX_LENGTHy, padding='post')

In [17]:
print(len(X_train1))
print(len(y_train1))
seq_length = X_train1.shape[1]
print("sequence length: ", seq_length)

12876
12876
sequence length:  21


In [18]:
ytest1.shape

(1600, 74)

In [19]:
# 3115
# 5512
# 8174
# 10667
# 12518
# 12706

In [21]:
def create_embedding_matrix( word_index, embedding_dim):
    vocab_size = len(word_index) + 1

    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    return embedding_matrix

embedding_dim = 200

embedding_matrixx = create_embedding_matrix( tokenizer.word_index, embedding_dim)
embedding_matrixy = create_embedding_matrix( tk.word_index, embedding_dim)

In [22]:
from keras import backend as K 
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed,Attention,Activation, dot,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

print("Size of vocabulary from the w2v model = {}".format(vocab_sizex))

K.clear_session()

latent_dim = 256
embedding_dim=200

# Encoder
encoder_inputs = Input(shape=(MAX_LENGTHx,))

#embedding layer
enc_emb =  Embedding(vocab_sizex, embedding_dim,weights=[embedding_matrixx],trainable=True,mask_zero=True)(encoder_inputs)

#encoder lstm 2
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.3,recurrent_dropout=0.3)
encoder_outputs, state_h, state_c= encoder_lstm3(enc_emb)

# Set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(vocab_sizey, embedding_dim,weights=[embedding_matrixy],trainable=True,mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.3,recurrent_dropout=0.3)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# ATTENTION layer
 
attn_layer = Attention()
attn_out = attn_layer([decoder_outputs,encoder_outputs])
decoder_concat_input = concatenate([decoder_outputs, attn_out])


#dense layer
decoder_dense =  TimeDistributed(Dense(vocab_sizey, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Size of vocabulary from the w2v model = 762
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 21)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 21, 200)      152400      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 200)    1000200     ['input_2[0][0]']                
                                                  

In [ ]:
model.compile(optimizer='rmsprop'
, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1,patience=2)
mc = ModelCheckpoint('best_model.hdf5', monitor = 'val_accuracy', verbose = 1, save_best_only = True, mode = 'max')
# Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.01)
# [MaskedCategoricalAccuracy(0),ExactMatchedAccuracy(0)]

In [ ]:
history=model.fit([X_train1,y_train1[:,:-1]], y_train1.reshape(y_train1.shape[0],y_train1.shape[1], 1)[:,1:] ,
                  epochs=50,callbacks=[es,mc]
                  ,batch_size=64, validation_split=0.2,shuffle=True,
                             workers=3, use_multiprocessing=True)

Epoch 1/50
161/161 [==============================] - ETA: 0s - loss: 1.1072 - accuracy: 0.2407
Epoch 00001: val_accuracy improved from -inf to 0.29100, saving model to best_model.hdf5
161/161 [==============================] - 563s 3s/step - loss: 1.1072 - accuracy: 0.2407 - val_loss: 1.8458 - val_accuracy: 0.2910
Epoch 2/50
161/161 [==============================] - ETA: 0s - loss: 0.7298 - accuracy: 0.4302
Epoch 00002: val_accuracy improved from 0.29100 to 0.38551, saving model to best_model.hdf5
161/161 [==============================] - 550s 3s/step - loss: 0.7298 - accuracy: 0.4302 - val_loss: 1.4704 - val_accuracy: 0.3855
Epoch 3/50
161/161 [==============================] - ETA: 0s - loss: 0.5894 - accuracy: 0.5071
Epoch 00003: val_accuracy improved from 0.38551 to 0.43223, saving model to best_model.hdf5
161/161 [==============================] - 551s 3s/step - loss: 0.5894 - accuracy: 0.5071 - val_loss: 1.2974 - val_accuracy: 0.4322
Epoch 4/50
161/161 [=======================

KeyboardInterrupt: ignored

In [23]:
model.load_weights('best_model.hdf5')

In [26]:
reverse_target_word_index=tk.index_word
reverse_source_word_index=tokenizer.index_word
target_word_index=tk.word_index

In [27]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(MAX_LENGTHx,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf = attn_layer([ decoder_outputs2,decoder_hidden_state_input])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [28]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']
    
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if(sampled_token_index==0):
            break
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (MAX_LENGTHy-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence


In [29]:
def seq2target(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [30]:
for i in range(11000,11005):
    print("Review:",relexicalize(train,seq2text(X_train1[i]),i))
    print("Original summary:",relexicalize(train,seq2target(y_train1[i]),i))
    print("Predicted summary:",relexicalize(train,decode_sequence(X_train1[i].reshape(1,MAX_LENGTHx)),i))
    print("\n")


Review: Allen_Forrest birth year 1981 Allen_Forrest birth place "Fort Campbell, KY, raised in Dothan, AL" Allen_Forrest background "solo_singer" Allen_Forrest genre Pop_music Allen_Forrest birth place "Fort Campbell, KY, raised in Dothan, AL" 
Original summary: Allen_Forrest is a solo Pop_music singer who was born in 1981 in "Fort Campbell, KY, raised in Dothan, AL" kentucky but was raised in dothan alabama 
Predicted summary:  Allen_Forrest was born in 1981 in "Fort Campbell, KY, raised in Dothan, AL" he plays Pop_music and he plays Pop_music music and is a solo singer


Review: Allen_Forrest birth year 1981 Allen_Forrest birth place Dothan,_Alabama Allen_Forrest genre Pop_music Allen_Forrest birth place Dothan,_Alabama Allen_Forrest background "solo_singer" 
Original summary: Allen_Forrest is an exponent of Pop_music and a solo singer who was born in the year 1981 in dothan Dothan,_Alabama alabama 
Predicted summary:  solo singer Allen_Forrest was born in Dothan,_Alabama in 1981 he p

In [31]:

for i in range(1200,1205):
    print("Review:",relexicalize(test,seq2text(xtest1[i]),i))
    print("Original summary:",test['entries'][i][str(i+1)]['lexicalisations'][0]['lex'])
    print("Predicted summary:",relexicalize(test,decode_sequence(xtest1[i].reshape(1,MAX_LENGTHx)),i))
    print("\n")

Review: Bhajji country India Bhajji currency Indian_rupee Bhajji leader name Narendra_Modi Bhajji leader name Narendra_Modi 
Original summary: Bhajji originates from India where the currency is the Indian rupee and the leaders are Narendra Modi and Sumitra Mahajan.
Predicted summary:  Bhajji is from India where the leader is the Indian rupee and the leader is Narendra_Modi and the currency is the Indian rupee


Review: Bhajji region Karnataka Bhajji main ingredients "Gram flour, vegetables" Bhajji alternative name "Bhaji, bajji" Bhajji ingredient Gram_flour 
Original summary: Bhajji, otherwise known as Bhaji or bajji, comes from the Karnataka region. The main ingredients are gram flour and vegetables.
Predicted summary:  Bhajji is from the Karnataka and main ingredients include red beans pork belly white rice ground meat chicharon fried egg plantain patacones chorizo arepa hogao sauce black pudding morcilla avocado and lemon


Review: Binignit ingredient Sweet_potato Binignit course De

In [32]:
for i in range(0,5):
    print("Review:",relexicalize(dev,seq2text(xdev1[i]),i))
    print("Original summary:",dev['entries'][i][str(i+1)]['lexicalisations'][0]['lex'])
    print("Predicted summary:",relexicalize(dev,decode_sequence(xdev1[i].reshape(1,MAX_LENGTHx)),i))
    print("\n")

Review: Aarhus leader name Jacob_Bundsgaard 
Original summary: The leader of Aarhus is Jacob Bundsgaard.
Predicted summary:  Aarhus's leader name is Jacob_Bundsgaard


Review: Aarhus_Airport runway length 2702.0 
Original summary: Aarhus Airport's runway length is 2702.0.
Predicted summary:  the runway length of Aarhus_Airport is 2702.0


Review: Adirondack_Regional_Airport elevation above the sea level (in metres) 507 
Original summary: Adirondack Regional Airport is 507 metres above sea level.
Predicted summary:  Adirondack_Regional_Airport is elevation above the sea level in metres k metres above sea level


Review: Adirondack_Regional_Airport location Harrietstown,_New_York 
Original summary: Adirondack Regional airport is located at Harrietstown, New York.
Predicted summary:  Adirondack_Regional_Airport is located in Harrietstown,_New_York


Review: Adolfo_Suárez_Madrid–Barajas_Airport location San_Sebastián_de_los_Reyes 
Original summary: Adolfo Suárez Madrid–Barajas Airport is f

In [40]:
from nltk.translate.bleu_score import sentence_bleu
def BLEU(data,input,k=-1):
 count=0
  if k==-1:
    k=len(data['entries'])
  for i in range(k):
    ref=[]
    for j in range(len(data['entries'][i][str(i+1)]['lexicalisations'])):
      tar=data['entries'][i][str(i+1)]['lexicalisations'][j]['lex']
      tar=tar.split()
      ref.append(tar)
    pridiction = relexicalize(data,decode_sequence(input[i].reshape(1,MAX_LENGTHx)),i).split()
    count=count+sentence_bleu(ref, pridiction)
  return count/k

In [41]:
c2=BLEU(test,xtest1)

In [42]:
c2

0.1228186356312705

In [43]:
c3=BLEU(dev,xdev1)

In [44]:
c3

0.11930895862149012

In [45]:
c1=BLEU(train,X_train1)
c1

0.1328589755694775